# Video Classifier Using CNN and RNN
#!dir

In [48]:
import os

# Define the directory path using double backslashes
directory_path = 'C:\\Users\\Hp\\Downloads\\signlanguage\\datasets\\train'
directory_path1 = 'C:\\Users\\Hp\\Downloads\\signlanguage\\datasets\\test'
files = os.listdir(directory_path)
files2=os.listdir(directory_path1)
# Print the list of files
print(files)
print(files2)


['hello', 'love', 'thumbs up']
['hello', 'love', 'thumbsup']


# Preparing Training Data

In [49]:
import os
import pandas as pd

# Define the directory path
directory_path = r'C:\Users\Hp\Downloads\signlanguage\datasets\train'

# Get all the directories in the main directory
subdirectories = os.listdir(directory_path)

rooms = []

# Iterate over each directory
for item in subdirectories:
    # Get the full path of the current directory
    item_path = os.path.join(directory_path, item)
    
    # Get all the file names in the current directory
    all_rooms = os.listdir(item_path)

    # Add them to the list with their corresponding directory
    for room in all_rooms:
        rooms.append((item, os.path.join(item_path, room)))

# Build a DataFrame        
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())


     tag                                         video_name
0  hello  C:\Users\Hp\Downloads\signlanguage\datasets\tr...
1  hello  C:\Users\Hp\Downloads\signlanguage\datasets\tr...
2  hello  C:\Users\Hp\Downloads\signlanguage\datasets\tr...
3  hello  C:\Users\Hp\Downloads\signlanguage\datasets\tr...
4  hello  C:\Users\Hp\Downloads\signlanguage\datasets\tr...
          tag                                         video_name
70  thumbs up  C:\Users\Hp\Downloads\signlanguage\datasets\tr...
71  thumbs up  C:\Users\Hp\Downloads\signlanguage\datasets\tr...
72  thumbs up  C:\Users\Hp\Downloads\signlanguage\datasets\tr...
73  thumbs up  C:\Users\Hp\Downloads\signlanguage\datasets\tr...
74  thumbs up  C:\Users\Hp\Downloads\signlanguage\datasets\tr...


In [50]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('train.csv')

# Preparing Test Data

In [51]:
import os
import pandas as pd

# Define the directory path
directory_path = r'C:\Users\Hp\Downloads\signlanguage\datasets\test'

# Get all the directories in the main directory
subdirectories = os.listdir(directory_path)

rooms = []

# Iterate over each directory
for item in subdirectories:
    # Get the full path of the current directory
    item_path = os.path.join(directory_path, item)
    
    # Get all the file names in the current directory
    all_rooms = os.listdir(item_path)

    # Add them to the list with their corresponding directory
    for room in all_rooms:
        rooms.append((item, os.path.join(item_path, room)))

# Build a DataFrame        
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())


     tag                                         video_name
0  hello  C:\Users\Hp\Downloads\signlanguage\datasets\te...
1  hello  C:\Users\Hp\Downloads\signlanguage\datasets\te...
2  hello  C:\Users\Hp\Downloads\signlanguage\datasets\te...
3  hello  C:\Users\Hp\Downloads\signlanguage\datasets\te...
4  hello  C:\Users\Hp\Downloads\signlanguage\datasets\te...
         tag                                         video_name
12  thumbsup  C:\Users\Hp\Downloads\signlanguage\datasets\te...
13  thumbsup  C:\Users\Hp\Downloads\signlanguage\datasets\te...
14  thumbsup  C:\Users\Hp\Downloads\signlanguage\datasets\te...
15  thumbsup  C:\Users\Hp\Downloads\signlanguage\datasets\te...
16  thumbsup  C:\Users\Hp\Downloads\signlanguage\datasets\te...


In [52]:
df = test_df.loc[:,['video_name','tag']]
df
df.to_csv('train.csv')

In [53]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [54]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

In [55]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)

Total videos for training: 17
Total videos for testing: 17


,Unnamed: 0,video_name,tag
14,14,C:\Users\Hp\Downloads\signlanguage\datasets\te...,thumbsup
1,1,C:\Users\Hp\Downloads\signlanguage\datasets\te...,hello
15,15,C:\Users\Hp\Downloads\signlanguage\datasets\te...,thumbsup
6,6,C:\Users\Hp\Downloads\signlanguage\datasets\te...,love
9,9,C:\Users\Hp\Downloads\signlanguage\datasets\te...,love
7,7,C:\Users\Hp\Downloads\signlanguage\datasets\te...,love
2,2,C:\Users\Hp\Downloads\signlanguage\datasets\te...,hello
0,0,C:\Users\Hp\Downloads\signlanguage\datasets\te...,hello
3,3,C:\Users\Hp\Downloads\signlanguage\datasets\te...,hello
13,13,C:\Users\Hp\Downloads\signlanguage\datasets\te...,thumbsup


In [56]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Read the training and testing data
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Define and initialize StringLookup layer with updated vocabulary
class_vocab = ['hello', 'love', 'thumbsup']
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=class_vocab)

# Encode labels
train_labels = label_processor(train_df["tag"].values).numpy()
test_labels = label_processor(test_df["tag"].values).numpy()

# Define constants
IMG_SIZE = 224
MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048
EPOCHS = 30

# Functions for processing videos
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y: start_y + min_dim, start_x: start_x + min_dim]

def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]  # Convert BGR to RGB
            frames.append(frame)
            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")

# Prepare videos and labels
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values.astype(str)

    labels = label_processor(labels).numpy()

    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for idx, path in enumerate(video_paths):
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH), dtype="bool")
        temp_frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
            temp_frame_mask[i, :length] = 1

        frame_features[idx] = temp_frame_features.squeeze()
        frame_masks[idx] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels

# Build the feature extractor model
feature_extractor = build_feature_extractor()

# Prepare data for training and testing
train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

# Print shapes of data
print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")
print(f"Train labels in train set: {train_labels.shape}")
print(f"Test labels in test set: {test_labels.shape}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

# The sequence model
Now, we can feed this data to a sequence model consisting of recurrent layers like GRU.

In [89]:
NUM_GRU_UNITS = 64
DROPOUT_RATE = 0.4
LEARNING_RATE = 0.001
OPTIMIZER = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
BATCH_SIZE = 64
EPOCHS = 100

# Modify the get_sequence_model function
def get_sequence_model(optimizer=OPTIMIZER):
    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    x = keras.layers.GRU(NUM_GRU_UNITS, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(NUM_GRU_UNITS // 2)(x)
    x = keras.layers.Dropout(DROPOUT_RATE)(x)
    x = keras.layers.Dense(NUM_GRU_UNITS // 2, activation=keras.layers.LeakyReLU(alpha=0.1))(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
    )

    return rnn_model

# Modify the run_experiment function
def run_experiment(optimizer=OPTIMIZER):
    print("Running experiment...")
    filepath = "./tmp/video_classifier.weights.h5"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model(optimizer=optimizer)
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    
    # Debugging steps
    # Check unique labels in training set
    unique_train_labels = np.unique(train_labels)
    print("Unique labels in training set:", unique_train_labels)

    # Check unique labels in testing set
    unique_test_labels = np.unique(test_labels)
    print("Unique labels in testing set:", unique_test_labels)

    if not np.all(np.isin(unique_test_labels, np.arange(len(class_vocab)))):
        print("Error: Testing set contains unexpected label values.")
        return

    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model

# Call the run_experiment function
print("Starting experiment...")
_, sequence_model = run_experiment()
print("Experiment completed.")



Starting experiment...
Running experiment...
Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5455 - loss: 0.9109
Epoch 1: val_loss improved from inf to 2.15437, saving model to ./tmp/video_classifier.weights.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.5455 - loss: 0.9109 - val_accuracy: 0.1667 - val_loss: 2.1544
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.7273 - loss: 0.6317
Epoch 2: val_loss did not improve from 2.15437
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.7273 - loss: 0.6317 - val_accuracy: 0.0000e+00 - val_loss: 2.5245
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.8182 - loss: 0.4692
Epoch 3: val_loss did not improve from 2.15437
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.8182 - loss: 0.4692 - val_accuracy: 0.0000e+00 - val_loss: 2.8022
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.9091 - loss: 0.3462
Epoch 4: val_loss did not improve from 2.15437
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110m

# Inference

In [90]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)


Test video path: C:\Users\Hp\Downloads\signlanguage\datasets\test\hello\v26.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 474ms/step
  hello: 54.49%
  love: 34.96%
  thumbsup: 10.54%
